In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import glob
from tqdm import tqdm
import pickle
from scipy import sparse
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from datetime import date, timedelta
import torch.optim as optim
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch_geometric import utils, data
from torch_geometric.loader import DataLoader
from sklearn.metrics import r2_score
pd.set_option('mode.chained_assignment',None)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
zones = [int(z[3:]) for z in pd.read_csv('data/processed/SimpleNNData.csv', index_col=0).filter(regex = 'lz').columns]

In [3]:
def make_PTG(graph, zones):
    attr, adj = graph

    # Filter out 
    if (attr.time_to_reservation.values[-1] >= 48) or ~attr.next_customer[-1]:
        return None
    
    if attr.leave_zone[-1] not in zones:
        return None

    # Slice
    _, labels = sparse.csgraph.connected_components(csgraph=adj, directed=False, return_labels=True)
    newl = labels[-1]
    indices = labels == newl   

    attr = attr[indices]
    adj = adj[indices,:].tocsc()[:,indices].tocsr()

    # Time variables
    attr['weekend'] = attr.time.dt.weekday//5

    def circle_transform(col, max_val=86400):
        tot_sec = ((col - col.dt.normalize()) / pd.Timedelta('1 second')).astype(int)
        cos_val = np.cos(2*np.pi*tot_sec/max_val)
        sin_val = np.sin(2*np.pi*tot_sec/max_val)
        return cos_val, sin_val

    attr['Time_Cos'], attr['Time_Sin'] = [x.values for x in circle_transform(attr.time)]

    # drop
    attr.drop(columns=['park_location_lat', 'park_location_long', 'leave_location_lat', 'leave_location_long', 'park_fuel', 'park_zone', 'moved', 'movedTF', 'time', 'prev_customer', 'next_customer'], inplace = True)

    # One hot encoding
    attr['leave_zone'] = pd.Categorical(attr['leave_zone'], categories=zones)
    attr = pd.get_dummies(attr, columns = ['leave_zone'], prefix='lz')

    attr['engine']= pd.Categorical(attr['engine'], categories=['118I', 'I3', 'COOPER', 'X1'])
    attr = pd.get_dummies(attr, columns = ['engine'], prefix='eng')

    # Normalize fuel and dist 
    attr['leave_fuel'] = attr['leave_fuel']/100
    #df['dist_to_station'] = df['dist_to_station']/5320

    # Get edges
    edge_index, edge_weight = utils.convert.from_scipy_sparse_matrix(adj)

    # Make pytorch data type
    d = data.Data(x = torch.tensor(attr.drop(columns = ['time_to_reservation']).to_numpy(dtype = 'float')).float(), edge_index=edge_index, edge_attr=edge_weight.float(), y = torch.tensor(attr.time_to_reservation.values).float())

    return d

In [76]:
files[0]

'data/processed/Graphs/20190901.pickle'

In [38]:
sdate = date(2019, 9, 1) # start date
delta = timedelta(days=1)
files = ['data/processed/Graphs/'+(sdate + timedelta(days=i)).strftime("%Y%m%d")+'.pickle' for i in range(delta.days + 1)]

with open(files[0], 'rb') as f:
    graph_collection = pickle.load(f)

In [39]:
attr, adj = next(iter(graph_collection.values()))

In [40]:
_, labels = sparse.csgraph.connected_components(csgraph=adj, directed=False, return_labels=True)
newl = labels[-1]
indices = labels == newl   

adj = adj[indices,:].tocsc()[:,indices].tocsr()

In [41]:
attr = attr[indices]
attr['degree'] = np.squeeze(np.asarray(adj.sum(axis=1)))/50
attr

,time,time_to_reservation,park_location_lat,park_location_long,leave_location_lat,leave_location_long,park_zone,leave_zone,park_fuel,leave_fuel,engine,moved,prev_customer,next_customer,movedTF,degree
car,,,,,,,,,,,,,,,,
WBY8P2104K7D80299,2019-08-25 18:46:49,349.338623,55.696625,12.553201,55.696625,12.553201,102443,102443,99,100,I3,0.0,True,False,False,0.447536
WBA1R5109K5N22621,2019-08-31 10:58:55,49.295834,55.704525,12.500069,55.704525,12.500069,102615,102615,78,78,118I,0.0,True,True,False,0.078930
WBA1R5103J7B13925,2019-08-31 13:16:32,47.377224,55.650139,12.512616,55.650139,12.512616,102751,102751,80,80,118I,0.0,False,True,False,0.067926
WBY1Z21010V307862,2019-08-31 13:59:57,10.709444,55.708710,12.559042,55.708710,12.559042,102414,102414,5,60,I3,0.0,True,True,False,0.240052
WBA1R5103J7B14248,2019-08-31 16:13:35,44.409443,55.634289,12.649727,55.634289,12.649727,185121,185121,73,73,118I,0.0,False,True,False,0.069351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WBY8P2106K7D93202,2019-08-31 23:57:53,1.540556,55.696636,12.516490,55.696636,12.516490,102554,102554,67,67,I3,0.0,False,True,False,0.150647
WBY8P2103K7D73456,2019-08-31 23:58:26,1.865278,55.681473,12.581757,55.681473,12.581757,102121,102121,33,33,I3,0.0,True,True,False,0.484341
WBY1Z21000V308050,2019-08-31 23:58:54,2.254444,55.708607,12.572359,55.708607,12.572359,102325,102325,58,58,I3,0.0,True,True,False,0.149996


,time,time_to_reservation,park_location_lat,park_location_long,leave_location_lat,leave_location_long,park_zone,leave_zone,park_fuel,leave_fuel,engine,moved,prev_customer,next_customer,movedTF
car,,,,,,,,,,,,,,,
WBY8P2104K7D80299,2019-08-25 18:46:49,349.338623,55.696625,12.553201,55.696625,12.553201,102443,102443,99,100,I3,0.0,True,False,False
WBA1R5109K5N22621,2019-08-31 10:58:55,49.295834,55.704525,12.500069,55.704525,12.500069,102615,102615,78,78,118I,0.0,True,True,False
WBA1R5103J7B13925,2019-08-31 13:16:32,47.377224,55.650139,12.512616,55.650139,12.512616,102751,102751,80,80,118I,0.0,False,True,False
WBY1Z21010V307862,2019-08-31 13:59:57,10.709444,55.708710,12.559042,55.708710,12.559042,102414,102414,5,60,I3,0.0,True,True,False
WBA1R5103J7B14248,2019-08-31 16:13:35,44.409443,55.634289,12.649727,55.634289,12.649727,185121,185121,73,73,118I,0.0,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WBY8P2106K7D93202,2019-08-31 23:57:53,1.540556,55.696636,12.516490,55.696636,12.516490,102554,102554,67,67,I3,0.0,False,True,False
WBY8P2103K7D73456,2019-08-31 23:58:26,1.865278,55.681473,12.581757,55.681473,12.581757,102121,102121,33,33,I3,0.0,True,True,False
WBY1Z21000V308050,2019-08-31 23:58:54,2.254444,55.708607,12.572359,55.708607,12.572359,102325,102325,58,58,I3,0.0,True,True,False


In [52]:
# Time variables
attr['weekend'] = attr.time.dt.weekday//5

def circle_transform(col, max_val=86400):
    tot_sec = ((col - col.dt.normalize()) / pd.Timedelta('1 second')).astype(int)
    cos_val = np.cos(2*np.pi*tot_sec/max_val)
    sin_val = np.sin(2*np.pi*tot_sec/max_val)
    return cos_val, sin_val

attr['Time_Cos'], attr['Time_Sin'] = [x.values for x in circle_transform(attr.time)]

# drop
attr.drop(columns=['park_location_lat', 'park_location_long', 'leave_location_lat', 'leave_location_long', 'park_fuel', 'park_zone', 'moved', 'movedTF', 'time', 'prev_customer', 'next_customer'], inplace = True)

# One hot encoding
attr['leave_zone'] = pd.Categorical(attr['leave_zone'], categories=zones)
attr = pd.get_dummies(attr, columns = ['leave_zone'], prefix='lz')

attr['engine']= pd.Categorical(attr['engine'], categories=['118I', 'I3', 'COOPER', 'X1'])
attr = pd.get_dummies(attr, columns = ['engine'], prefix='eng')

# Normalize fuel and dist 
attr['leave_fuel'] = attr['leave_fuel']/100

In [55]:
attr.drop(columns = ['time_to_reservation'])

,leave_fuel,weekend,Time_Cos,Time_Sin,lz_102111,lz_102121,lz_102122,lz_102131,lz_102141,lz_102142,...,lz_185132,lz_185141,lz_185142,lz_185143,lz_185154,lz_185203,eng_118I,eng_I3,eng_COOPER,eng_X1
car,,,,,,,,,,,,,,,,,,,,,
WBA1R5108J5K57642,0.36,0,0.942908,0.333053,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
WMWXU7106KTM90596,0.82,1,0.793486,0.608588,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
WMWXR3103KTK68690,0.28,0,0.837957,0.545736,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
WBY8P2107K7D92186,0.56,0,-0.438567,0.898698,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
WBY8P210XK7D96376,0.58,0,-0.497416,0.867512,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WBY8P2104K7D98348,1.00,1,0.999950,-0.010035,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
WMWXU710XKTM91735,0.57,1,0.999961,-0.008872,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
WMWXU7106KTM91411,0.55,1,0.999962,-0.008727,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
sdate = date(2019, 9, 15)   # start date
edate = date(2019, 9, 17)   # end date
delta = edate - sdate       # as timedelta
files = ['data/processed/Graphs/'+(sdate + timedelta(days=i)).strftime("%Y%m%d")+'.pickle' for i in range(delta.days + 1)]

dataset = []

with open(files[0], 'rb') as f:
    graph_collection = pickle.load(f)

for g in graph_collection.values():
    res = make_PTG(g,zones)
    if res:
        dataset.append(res)

train_val_size = int(0.8 * len(dataset))
val_test_size = len(dataset)-train_val_size
train_val_data, test_data = torch.utils.data.random_split(dataset, [train_val_size, val_test_size])
train_size = train_val_size-val_test_size
train_data, val_data = torch.utils.data.random_split(train_val_data, [train_size, val_test_size])
del train_val_data

for file in tqdm(files[1:]):
    dataset = []
    with open(file, 'rb') as f:
        graph_collection = pickle.load(f)

    for g in graph_collection.values():
        res = make_PTG(g,zones)
        if res:
            dataset.append(res)

    train_val_size = int(0.8 * len(dataset))
    val_test_size = len(dataset)-train_val_size
    train_val_data_tmp, test_data_tmp = torch.utils.data.random_split(dataset, [train_val_size, val_test_size])
    train_size = train_val_size-val_test_size
    train_data_tmp, val_data_tmp = torch.utils.data.random_split(train_val_data_tmp, [train_size, val_test_size])

    train_data = torch.utils.data.ConcatDataset([train_data,train_data_tmp])
    val_data = torch.utils.data.ConcatDataset([val_data,val_data_tmp])
    test_data = torch.utils.data.ConcatDataset([test_data,test_data_tmp])

#del train_val_data_tmp, test_data_tmp, train_data_tmp, val_data_tmp, dataset

100%|██████████| 2/2 [00:59<00:00, 29.99s/it]


In [19]:
train_data

In [9]:
sdate = date(2019, 8, 15)   # start date
edate = date(2019, 8, 20)   # end date
delta = edate - sdate       # as timedelta
files = ['data/processed/Graphs/'+(sdate + timedelta(days=i)).strftime("%Y%m%d")+'.pickle' for i in range(delta.days + 1)]

dataset = []

for file in tqdm(files):
    with open(file, 'rb') as f:
        graph_collection = pickle.load(f)

    for g in graph_collection.values():
        res = make_PTG(g,zones)
        if res:
            dataset.append(res.to(device))

100%|██████████| 6/6 [03:43<00:00, 37.21s/it]


In [10]:
train_val_size = int(0.8 * len(dataset))
val_test_size = len(dataset)-train_val_size
train_val_data, test_data = torch.utils.data.random_split(dataset, [train_val_size, val_test_size])
train_size = train_val_size-val_test_size
train_data, val_data = torch.utils.data.random_split(train_val_data, [train_size, val_test_size])
del dataset

In [61]:
len(train_data)//32

134

In [41]:
t.x

tensor([[ 0.3600,  0.0000,  0.9429,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.8200,  1.0000,  0.7935,  ...,  0.0000,  1.0000,  0.0000],
        [ 0.2800,  0.0000,  0.8380,  ...,  0.0000,  1.0000,  0.0000],
        ...,
        [ 1.0000,  1.0000, -0.7238,  ...,  1.0000,  0.0000,  0.0000],
        [ 0.5400,  1.0000, -0.7267,  ...,  1.0000,  0.0000,  0.0000],
        [ 0.4300,  1.0000, -0.7269,  ...,  1.0000,  0.0000,  0.0000]])

In [37]:
from torch_scatter import scatter
def pool_neighbor(x, edge_index, num_nodes, reduce = 'mean'):
    r"""Average pools neighboring node features, where each feature in
    :obj:`data.x` is replaced by the average feature values from the central
    node and its neighbors.
    """

    row, col = edge_index
    row, col = (row, col)

    x = scatter(x[row], col, dim=0, dim_size=num_nodes, reduce=reduce)
    return x

In [26]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

In [28]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(269, 32)
        self.conv2 = GCNConv(32, 1)
        #self.conv3 = GCNConv(32, 1)

    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr

        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, p = 0.2, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        #x = F.relu(x)
        #x = self.conv3(x, edge_index, edge_weight)

        return x.squeeze()

GNN = GCN().to(device)
print(GNN, sum(p.numel() for p in GNN.parameters()))

GCN(
  (conv1): GCNConv(269, 32)
  (conv2): GCNConv(32, 1)
) 8673


In [30]:
optimizer = optim.Adam(GNN.parameters(), lr=0.03, weight_decay = 0.0001) #Chaged to Adam and learning + regulariztion rate set
criterion = nn.MSELoss(reduction = 'mean')

In [31]:
# Set number of epochs
num_epochs = 3

# Set up lists for loss/R2
train_r2, train_loss = [], []
valid_r2, valid_loss = [], []
cur_loss = 0
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    ### Train
    cur_loss_train = []
    GNN.train()
    for batch in train_loader:
        optimizer.zero_grad()
        out = GNN(batch)
        batch_loss = criterion(out[batch.ptr[1:]-1],batch.y[batch.ptr[1:]-1])
        batch_loss.backward()
        optimizer.step()

        cur_loss_train.append(batch_loss.item())
    
    train_losses.append(np.mean(cur_loss_train))

    ### Evaluate training
    GNN.eval()
    train_preds, train_targs = [], []
    for batch in train_loader:
        preds = GNN(batch)
        train_targs += list(batch.y.numpy()[batch.ptr[1:]-1])
        train_preds += list(preds.detach().numpy()[batch.ptr[1:]-1])


    ### Evaluate validation
    val_preds, val_targs = [], []
    cur_loss_val = []
    for batch in val_loader:
        preds = GNN(batch)[batch.ptr[1:]-1]
        y_val = batch.y[batch.ptr[1:]-1]
        val_targs += list(y_val.numpy())
        val_preds += list(preds.detach().numpy())
        cur_loss_val.append(criterion(preds, y_val).item())

    val_losses.append(np.mean(cur_loss_val))


    train_r2_cur = r2_score(train_targs, train_preds)
    valid_r2_cur = r2_score(val_targs, val_preds)
    
    train_r2.append(train_r2_cur)
    valid_r2.append(valid_r2_cur)

    print("Epoch %2i: Train Loss %f , Valid Loss %f ,Train R2 %f, Valid R2 %f" % (
                epoch+1, train_losses[-1], val_losses[-1],train_r2_cur, valid_r2_cur))

Epoch  1: Train Loss 18.009013 , Valid Loss 15.885797 ,Train R2 0.207517, Valid R2 0.165569
Epoch  2: Train Loss 17.043701 , Valid Loss 15.617674 ,Train R2 0.226370, Valid R2 0.179597
Epoch  3: Train Loss 16.611401 , Valid Loss 15.763800 ,Train R2 0.245041, Valid R2 0.171937


In [69]:
# setting hyperparameters and gettings epoch sizes
batch_size = 4084
num_epochs = 121
num_samples_train = X_train.shape[0]
num_batches_train = num_samples_train // batch_size
num_samples_valid = X_val.shape[0]
num_batches_valid = num_samples_valid // batch_size

# setting up lists for handling loss/accuracy
train_r2, train_loss = [], []
valid_r2, valid_loss = [], []
test_acc, test_loss = [], []
cur_loss = 0
train_losses = []
val_losses = []

get_slice = lambda i, size: range(i * size, (i + 1) * size)

early_stopping = EarlyStopping(patience=10, verbose=False)

for epoch in range(num_epochs):
    # Forward -> Backprob -> Update params
    ## Train
    cur_loss_train = []
    net.train()
    for i in range(num_batches_train):
        optimizer.zero_grad()
        slce = get_slice(i, batch_size)
        output = net(X_train[slce])
        
        # compute gradients given loss
        target_batch = y_train[slce]
        batch_loss = criterion(output, target_batch)
        batch_loss.backward()
        optimizer.step()
        
        cur_loss_train.append(batch_loss.item())
    train_losses.append(np.mean(cur_loss_train))

    net.eval()
    ### Evaluate training
    train_preds, train_targs = [], []
    for i in range(num_batches_train):
        slce = get_slice(i, batch_size)
        output = net(X_train[slce])
        
        preds = output
        
        train_targs += list(y_train[slce].numpy())
        train_preds += list(preds.data.numpy())
    
    ### Evaluate validation
    val_preds, val_targs = [], []
    cur_loss_val = []
    for i in range(num_batches_valid):
        slce = get_slice(i, batch_size)
        
        output = net(X_val[slce])
        preds = output
        val_targs += list(y_val[slce].numpy())
        val_preds += list(preds.data.numpy())

        cur_loss_val.append(criterion(output, y_val[slce]).item())

    val_losses.append(np.mean(cur_loss_val))


    train_r2_cur = r2_score(train_targs, train_preds)
    valid_r2_cur = r2_score(val_targs, val_preds)
    
    train_r2.append(train_r2_cur)
    valid_r2.append(valid_r2_cur)

    # EarlyStopping
    early_stopping(val_losses[-1], net)
    if early_stopping.early_stop:
        print("Early stopping")
        print("Epoch %2i: Train Loss %f , Valid Loss %f , Train R2 %f, Valid R2 %f" % (
            epoch+1, train_losses[-1], val_losses[-1], train_r2_cur, valid_r2_cur))
        break
    
    if epoch % 10 == 0:
        print("Epoch %2i: Train Loss %f , Valid Loss %f ,Train R2 %f, Valid R2 %f" % (
                epoch+1, train_losses[-1], val_losses[-1],train_r2_cur, valid_r2_cur))

epoch = np.arange(len(train_r2))
plt.figure()
plt.plot(epoch, train_r2, 'r', epoch, valid_r2, 'b')
plt.legend(['Train Accucary','Validation Accuracy'])
plt.xlabel('Updates'), plt.ylabel('R2')

NameError: name 'X_train' is not defined

In [2]:
#Get files
sdate = date(2019, 8, 15)   # start date
edate = date(2019, 10, 2)   # end date
delta = edate - sdate       # as timedelta
files = ['graphs/'+(sdate + timedelta(days=i)).strftime("%Y%m%d")+'.pickle' for i in range(delta.days + 1)]

In [7]:
files[0][11:15]

'0815'